In [ ]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath("..")  # adjust if needed
if project_root not in sys.path:
    sys.path.insert(-1, project_root)
print(f"Project root added to sys.path: {project_root}")
import torch
#from .ViTSubmodules import *

from models.quantized.quant_ready_LSTMNetVIT import QuantReadyLSTMNetViT
from models.quantized.quant_ready_ITAConformerLSTM import QuantReadyITALSTM
from models.ITAConformerLSTM import ITALSTM



# 1 Prepering the pretrained weights
We need to remove the spectral_norm wrappers from the models since they arent supported by the torch QAT API

We start by loading the model 

In [ ]:
model = ITALSTM()
model.load_state_dict(torch.load("../models/pretrained_models/ITALSTM.pth", map_location='cpu'))
model.eval()

Then we need to generate a dummy input for a single forward pass that normalises the weights

In [ ]:
def generate_dummy_input(traj_len=10):
    # Simulate a sequence of depth images (T, 1, 60, 90)
    depth_images = torch.randn(traj_len, 1, 60, 90)

    # Simulate desired velocities (T, 1)
    control_input = torch.rand(traj_len, 1)

    # Simulate quaternion orientations (T, 4)
    orientation = torch.tensor([[1.0, 0.0, 0.0, 0.0]] * traj_len).float()

    return [depth_images, control_input, orientation]

In [ ]:
x = generate_dummy_input()
# Perform a forward pass to normalize the weights
with torch.no_grad():
    _ = model(x)

Now we can remove the spectral norm wrappers and save the model

In [ ]:
torch.nn.utils.remove_spectral_norm(model.decoder)  
torch.nn.utils.remove_spectral_norm(model.nn_fc2)


torch.save(model.state_dict(), "../models/pretrained_models/checkpoints_for_qat/ITALSTM.pth")

In [ ]:

model = QuantReadyITALSTM()
pretrained_weights = torch.load("../models/pretrained_models/checkpoints_for_qat/ITALSTM.pth", map_location='cpu')
# Load the pretrained weights
model.load_state_dict(pretrained_weights, strict=False)
